<a href="https://colab.research.google.com/github/BFvandy/Nissan-Text-Mining-POC-with-Transformer/blob/main/Reddit_Zeroshot_after_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = "/content/drive/My Drive/Nissan Internship/Reddit Comments"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/Nissan Internship/Reddit Comments


# Load Transformer

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 81.2 MB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

# Load Dataset & adjujst structure

In [ ]:
comments = pd.read_csv('reddit_cmt_prefiltered.csv', encoding_errors='ignore')

In [ ]:
test_df = comments.copy()

In [ ]:
comments.head(10)

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Rogue,Reddit,This is incredible timing for me as my cvt jus...,1.659024e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Nissan,Rogue,Reddit,Op do you own it out right? i'd sell it or tra...,1.659036e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nissan,Rogue,Reddit,I have 2019 sport transmission failed at 42k m...,1.659033e+09,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Nissan,Rogue,Reddit,Not yet. but i bought an extended warranty any...,1.659030e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Nissan,Rogue,Reddit,My 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Nissan,Rogue,Reddit,"2015 altima . 80k, cvt still strong.",1.659045e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Nissan,Rogue,Reddit,"Yep, too bad i'm past the mileage already",1.659056e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Nissan,Rogue,Reddit,Cvt needs to go extinct,1.659062e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Nissan,Rogue,Reddit,"I should've god 1500 warranty for ""100k miles""...",1.659056e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Nissan,Rogue,Reddit,I have a 2003 xterra with a lil less than 150k...,1.659062e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
comments.dtypes

Make                         object
Model                        object
Source                       object
Text                         object
timestamp                   float64
dependable                  float64
lasts long                  float64
value for money             float64
quality fit and finish      float64
attractive styling          float64
safe                        float64
retains resale value        float64
driver comfort              float64
fun to drive                float64
advanced features           float64
responsive handling         float64
prestigious                 float64
dealerships                 float64
fuel efficient              float64
quick acceleration          float64
environmentally friendly    float64
affordable                  float64
trusted                     float64
leader                      float64
responsible                 float64
confident                   float64
innovative                  float64
exciting                    

In [ ]:
comments['Text'][2]

"I have 2019 sport transmission failed at 42k miles i have warranty till 100k but yea already replaced as soon as warranty goes car goesi had a 2015 maxima for 5 years cvtfailed at 110k miles serviced car by book transmission every 25k dealership told me hey at least you got 100 k out of it it ok my boss told me he going to buy new car and gift me his truck it only has 36k and it's reliable it's a lexus"

In [ ]:
sequence_to_classify = "I have 2019 sport transmission failed at 42k miles i have warranty till 100k but yea already replaced as soon as warranty goes car goesi had a 2015 maxima for 5 years cvtfailed at 110k miles serviced car by book transmission every 25k dealership told me hey at least you got 100 k out of it it ok my boss told me he going to buy new car and gift me his truck it only has 36k and it's reliable it's a lexus"
candidate_labels = ['dependable', 'not dependable']
sas = classifier(sequence_to_classify, candidate_labels, multi_label=False)

In [ ]:
sas

{'labels': ['not dependable', 'dependable'],
 'scores': [0.8121617436408997, 0.18783828616142273],
 'sequence': "I have 2019 sport transmission failed at 42k miles i have warranty till 100k but yea already replaced as soon as warranty goes car goesi had a 2015 maxima for 5 years cvtfailed at 110k miles serviced car by book transmission every 25k dealership told me hey at least you got 100 k out of it it ok my boss told me he going to buy new car and gift me his truck it only has 36k and it's reliable it's a lexus"}

In [ ]:
ind = sas['labels'].index('dependable')
ind

1

In [ ]:
sas.get('scores')[ind]

0.18783828616142273

In [ ]:
#all the labels
# 17 functional attributes
dependable_labels = ['dependable', 'not dependable']
lasts_labels = ['lasts long', 'not lasts long']
value_labels = ['value for money', 'not value for money']
quality_labels = ['quality fit and finish', 'no quality fit and finish']
attractive_labels = ['attractive styling', 'not attractive styling']
safe_labels = ['safe', 'not safe']
retains_labels	= ['retains resale value', 'not retains resale value']
driver_labels	= ['driver comfort', 'driver not comfortable']
fun_labels = ['fun to drive', 'not fun to drive']
advanced_labels = ['advanced features', 'no advanced features']
responsive_labels = ['responsive handling', 'not responsive handling']
prestigious_labels = ['prestigious	', 'not prestigious	']
dealerships_labels = ['good dealerships', 'bad dealerships']
fuel_labels = ['fuel efficient', 'not fuel efficient']
quick_labels = ['quick acceleration', 'slow acceleration']
environ_labels = ['environmentally friendly', 'not environmentally friendly']
affordable_labels = ['affordable', 'not affordable']

# 12 personality attributes
trust_label = ['trusted', 'not trusted']
leader_label = ['leader', 'not leader']
resp_label = ['responsible', 'not responsible']
conf_label = ['confident','not confident']
innov_label = ['innovative', 'not innovative'] 
excit_label = ['exciting', 'not exciting']
pract_label = ['practical', 'not practical']
adven_label = ['adventurous', 'not adventurous']
pass_label = ['passionate', 'not passionate']
dist_label = ['distinctive','not distinctive']
youth_label = ['youthful', 'not youthful']
aggre_label = ['aggressive', 'not aggressive']


In [ ]:
funct_labels = [dependable_labels, lasts_labels, value_labels, quality_labels, attractive_labels, safe_labels, retains_labels, driver_labels, fun_labels, advanced_labels, responsive_labels, prestigious_labels, dealerships_labels, fuel_labels, quick_labels, environ_labels, affordable_labels]
pers_labels = [trust_label, leader_label, resp_label, conf_label, innov_label, excit_label, pract_label, adven_label, pass_label, dist_label, youth_label, aggre_label]

In [ ]:
all_labels = funct_labels + pers_labels

In [ ]:
all_labels

[['dependable', 'not dependable'],
 ['lasts long', 'not lasts long'],
 ['value for money', 'not value for money'],
 ['quality fit and finish', 'no quality fit and finish'],
 ['attractive styling', 'not attractive styling'],
 ['safe', 'not safe'],
 ['retains resale value', 'not retains resale value'],
 ['driver comfort', 'driver not comfortable'],
 ['fun to drive', 'not fun to drive'],
 ['advanced features', 'no advanced features'],
 ['responsive handling', 'not responsive handling'],
 ['prestigious\t', 'not prestigious\t'],
 ['good dealerships', 'bad dealerships'],
 ['fuel efficient', 'not fuel efficient'],
 ['quick acceleration', 'slow acceleration'],
 ['environmentally friendly', 'not environmentally friendly'],
 ['affordable', 'not affordable'],
 ['trusted', 'not trusted'],
 ['leader', 'not leader'],
 ['responsible', 'not responsible'],
 ['confident', 'not confident'],
 ['innovative', 'not innovative'],
 ['exciting', 'not exciting'],
 ['practical', 'not practical'],
 ['adventurous',

In [ ]:
len(all_labels)

29

# Apply Zero-shot model

In [ ]:
for i in range(len(comments)):
  for j in range(29):
    #29 labels in total
    column_num = j + 5
    #match column index with label index
    if comments.iat[i, column_num] != 0:
      try:
        res = classifier(comments.at[i, 'Text'], all_labels[j], multi_label=False)
        ind = res['labels'].index(all_labels[j][0])
        comments.iat[i, column_num] = res.get('scores')[ind]
      except:
        pass
  if i % 1000 == 0:
    print(i)
  else:
    pass

0


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [ ]:
comments

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Rogue,Reddit,This is incredible timing for me as my cvt jus...,1.659024e+09,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Nissan,Rogue,Reddit,Op do you own it out right? i'd sell it or tra...,1.659036e+09,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nissan,Rogue,Reddit,I have 2019 sport transmission failed at 42k m...,1.659033e+09,0.187838,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Nissan,Rogue,Reddit,Not yet. but i bought an extended warranty any...,1.659030e+09,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Nissan,Rogue,Reddit,My 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,0.310584,0.000000,0.0,0.0,0.0,...,0.978874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28376,Toyota,RAV4,Reddit,"I would maybe wait for the fall, just so your ...",1.625451e+09,0.000000,0.863397,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28377,Toyota,RAV4,Reddit,Yeah i probably will just wait until september...,1.625464e+09,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28378,Toyota,RAV4,Reddit,Change it.,1.637008e+09,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28379,Toyota,RAV4,Reddit,Did plenty of drain and fills on higher mileag...,1.637016e+09,0.938793,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
comments.groupby('Model').mean()

,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
Model,,,,,,,,,,,,,,,,,,,,,
Bronco Sport,1.622676e+09,0.015624,0.031572,0.009578,0.025121,0.019868,0.006575,0.006774,0.011076,0.015259,...,0.043342,0.010600,0.011751,0.022647,0.029864,0.023680,0.020947,0.037137,0.005596,0.017698
CR-V,1.625946e+09,0.027420,0.035349,0.015369,0.025037,0.011540,0.011609,0.008272,0.016053,0.012710,...,0.057229,0.009141,0.010852,0.012685,0.037612,0.019281,0.013406,0.040152,0.004458,0.022469
CX-5,1.644331e+09,0.034579,0.049620,0.022104,0.043173,0.026281,0.016714,0.012235,0.022760,0.018723,...,0.068599,0.015207,0.017726,0.024329,0.044314,0.020549,0.019949,0.051189,0.007363,0.024335
Equinox,1.582750e+09,0.024834,0.031953,0.014989,0.034689,0.026300,0.014316,0.012199,0.021650,0.009032,...,0.085063,0.010345,0.018523,0.018610,0.042881,0.023244,0.018122,0.044712,0.007032,0.019388
Escape,1.638727e+09,0.023835,0.042299,0.018465,0.023325,0.014849,0.009780,0.010080,0.014348,0.008529,...,0.077633,0.011707,0.019213,0.018880,0.038593,0.021159,0.017317,0.048224,0.006400,0.023232
Forester,1.652661e+09,0.019924,0.031407,0.011572,0.025596,0.014570,0.010022,0.009872,0.015290,0.010364,...,0.056459,0.009249,0.014115,0.015550,0.040077,0.020666,0.017957,0.042444,0.006075,0.021972
RAV4,1.647236e+09,0.039000,0.058917,0.026355,0.038790,0.019885,0.019641,0.017278,0.026035,0.016081,...,0.078741,0.014763,0.018841,0.022546,0.057880,0.022830,0.018759,0.043099,0.006621,0.028783
Rogue,1.631547e+09,0.032764,0.050125,0.011881,0.027749,0.011363,0.011967,0.011336,0.021075,0.010689,...,0.074297,0.008638,0.015674,0.014151,0.042112,0.017375,0.017767,0.041863,0.005780,0.020310
Sportage,1.655513e+09,0.019649,0.051184,0.023938,0.030337,0.020942,0.010879,0.013846,0.017925,0.013405,...,0.057995,0.011581,0.016479,0.021643,0.036731,0.022489,0.022447,0.041221,0.006750,0.024941


In [ ]:
comments.to_csv('reddit_score_v1.csv', index=False)